In [1]:
import numpy as np
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model

In [2]:
# load mnist dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [4]:
# from sparse label to categorical (one-hot vector)
num_labels = len(np.unique(y_train))
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [5]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [6]:
# reshape and normalize input images
image_size = x_train.shape[1]
x_train = np.reshape(x_train, [-1, image_size, image_size, 1])
x_test  = np.reshape(x_test, [-1, image_size, image_size, 1])
x_train = x_train.astype('float32') / 255
x_test  = x_test.astype('float32') / 255

In [7]:
# network params
input_shape = (image_size, image_size, 1)
batch_size = 128
dropout = 0.3
filters = 64
kernel_size = 3

In [8]:
# use functional API to build CNN layers

inputs = Input(shape=input_shape)
y = Conv2D(filters=filters, 
         kernel_size=kernel_size, 
         activation='relu')(inputs)
y = MaxPooling2D()(y)
y = Conv2D(filters=filters, 
         kernel_size=kernel_size, 
         activation='relu')(y)
y = MaxPooling2D()(y)
y = Conv2D(filters=filters, 
         kernel_size=kernel_size, 
         activation='relu')(y)

# image to vector before connectiong to Dense layer
y = Flatten()(y)
# dropout regularization
y = Dropout(dropout)(y)

outputs = Dense(num_labels, activation='softmax')(y)

# build the model by supplying inputs/ outputs
model = Model(inputs=inputs, outputs=outputs)

# network model in text
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928 

In [9]:
# classifier loss, optimizer, accuracy
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [10]:
# train the model
model.fit(x_train, y_train, 
          validation_data = (x_test, y_test), 
          epochs=20, 
          batch_size=batch_size)

Epoch 1/20
469/469 [==============================] - 25s 51ms/step - loss: 0.2819 - accuracy: 0.9112 - val_loss: 0.0653 - val_accuracy: 0.9792
Epoch 2/20
469/469 [==============================] - 24s 50ms/step - loss: 0.0731 - accuracy: 0.9773 - val_loss: 0.0386 - val_accuracy: 0.9876
Epoch 3/20
469/469 [==============================] - 23s 48ms/step - loss: 0.0534 - accuracy: 0.9837 - val_loss: 0.0309 - val_accuracy: 0.9897
Epoch 4/20
469/469 [==============================] - 25s 54ms/step - loss: 0.0429 - accuracy: 0.9863 - val_loss: 0.0271 - val_accuracy: 0.9914
Epoch 5/20
469/469 [==============================] - 24s 52ms/step - loss: 0.0372 - accuracy: 0.9881 - val_loss: 0.0288 - val_accuracy: 0.9902
Epoch 6/20
469/469 [==============================] - 24s 51ms/step - loss: 0.0311 - accuracy: 0.9906 - val_loss: 0.0290 - val_accuracy: 0.9902
Epoch 7/20
469/469 [==============================] - 24s 51ms/step - loss: 0.0277 - accuracy: 0.9914 - val_loss: 0.0239 - val_accuracy:

In [11]:
# evaluate the model
score = model.evaluate(x_test, y_test, 
                       batch_size=batch_size, 
                       verbose=0)

print("\nTest accuracy: %.1f%%" % (100 * score[1]))


Test accuracy: 99.4%
